# RAG application built on IAG PDS

In [1]:
import os

# Define the path to your local project folder
persistent_folder = "/Users/pauline/Desktop/github_repos/rag-ai-chatbot/"

# Create the folder if it doesn't exist
os.makedirs(persistent_folder, exist_ok=True)

# Change directory to the project folder
os.chdir(persistent_folder)

print(f"Current working directory: {os.getcwd()}")

Current working directory: /Users/pauline/Desktop/github_repos/rag-ai-chatbot


In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("iag_pds.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
#data

In [3]:
len(data)

74

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  270


In [5]:
docs[7]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Macintosh)', 'creationdate': '2016-05-10T14:59:33+12:00', 'moddate': '2017-02-24T17:04:34+11:00', 'trapped': '/False', 'source': 'iag_pds.pdf', 'total_pages': 74, 'page': 3, 'page_label': '3'}, page_content='will not receive any compensation, and you have no right to take action against IAG if \nany of your Notes are Written-Off \nReceipt of Ordinary Shares on \nConversion\nYou will receive approximately NZ$1.01 worth of Ordinary Shares for each of your \nNotes that is Converted, unless a cap referred to as the “Maximum Conversion \nNumber” applies – see Section 6.7 of this PDS (Conversion formulae). It is likely the \nMaximum Conversion Number will apply following a Non-Viability Trigger Event, \nin which case you may receive significantly less than NZ$1.01 worth of Ordinary \nShares for each of your Notes that is Converted')

In [6]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

#Get an API key:
# Head to https://ai.google.dev/gemini-api/docs/api-key to generate a Google AI API key. Paste in .env file

# Embedding models: https://python.langchain.com/v0.1/docs/integrations/text_embedding/

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [7]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [8]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What is a Non-Viability Trigger Event?")


In [9]:
len(retrieved_docs)

10

In [10]:
print(retrieved_docs[5].page_content)

• IAG would be required on the next payment due on the Notes to pay an additional amount in 
respect of a withholding or deduction on account of taxes on a payment on the Notes;
• IAG or the consolidated tax group of which it is a member would be exposed to more than a de 
minimis amount of other taxes, assessments or other governmental charges in connection with 
the Notes; or
• IAG determines that any interest payable on the Notes is not or may not be allowed as a 
deduction for the purposes of Australian income tax,
provided that on the Issue Date, IAG did not expect that matters giving rise to the Tax Event  
would occur
Tier 1 Capital Tier 1 capital (as defined by APRA from time to time)
Tier 2 Capital Tier 2 capital (as defined by APRA from time to time)
Trigger Event Date the date on which APRA notifies IAG that a Non-Viability Trigger Event has occurred
Trust Deed the trust deed dated 4 May 2016 between IAG and the Supervisor, as amended from time to time.


In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",temperature=0.3, max_tokens=500)

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [13]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [14]:
response = rag_chain.invoke({"input": "What is a Non-Viability Trigger Event?"})
print(response["answer"])

A Non-Viability Trigger Event occurs if APRA determines that IAG would become non-viable without converting or writing off capital instruments (like Notes) or receiving a public sector capital injection.  This event is triggered at APRA's discretion and may include, but isn't limited to, serious impairment of IAG’s financial position and solvency.  Several factors, including macroeconomic conditions and the financial health of IAG's subsidiaries, can impact IAG's viability.


In [ ]:
import mesop as me
import mesop.labs as mel

me.colab_run()


Running server on: http://localhost:32123


In [ ]:
# Define the chat function and connect to RAG model
@me.page(path="/chat")
def chat():
    mel.chat(transform)

# Modify transform to use RAG to generate responses
def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
    # Assuming rag_chain is your RAG pipeline
    response = rag_chain.invoke({"input": prompt})
    return response["answer"]  # Return the RAG model's answer

# Display the chat window in Colab
me.colab_show(path="/chat")


<IPython.core.display.Javascript object>

In [ ]:
import mesop as me
import mesop.labs as mel

# Initialize Mesop as usual
me.colab_run()

# Define the chat UI page and connect it to the RAG model
@me.page(path="/chat")
def chat():
    mel.chat(transform)

# Modify transform to use the RAG model to generate responses
def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
    # Assuming rag_chain is your RAG pipeline
    response = rag_chain.invoke({"input": prompt})
    return response["answer"]  # Return the RAG model's answer

# Start the Mesop server on port 8080
me.run(port=8080)



Running server on: http://localhost:32123
 * Serving Flask app 'mesop.server.server'
 * Debug mode: off


Address already in use
Port 32123 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: module 'mesop' has no attribute 'run'